# Dam Breach Results Extraction and Sensitivity Analysis

This notebook demonstrates:
1. **Extracting baseline breach results** from HDF files
2. **Reading breach parameters** from plan files
3. **Modifying parameters iteratively** (one parameter at a time)
4. **Comparing results** across different scenarios
5. **Visualizing sensitivity** to parameter changes

**Project:** BaldEagleCrkMulti2D (HEC-RAS Example)  
**Baseline Plan:** 02  
**Version:** 6.6

**Workflow:**
- Extract baseline results
- Clone plan and modify one parameter
- Re-extract results and compare
- Repeat for multiple parameters
- Plot all scenarios together

## Setup and Imports

In [ ]:
# Standard imports
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import shutil

# Add ras-commander to path (if running from examples folder)
repo_root = Path.cwd().parent
if repo_root not in sys.path:
    sys.path.append(str(repo_root))

# Import ras-commander
from ras_commander import (
    init_ras_project,
    RasExamples,
    RasBreach,
    ras
)

print("Imports successful!")
print(f"Working directory: {Path.cwd()}")

## 1. Extract and Initialize Project

In [ ]:
# Extract BaldEagleCrkMulti2D example project
project_path = RasExamples.extract_project("BaldEagleCrkMulti2D")
print(f"Extracted project to: {project_path}")

# Initialize the project
init_ras_project(project_path, "6.6")
print(f"\nInitialized project: {ras.project_name}")
print(f"\nAvailable plans:")
print(ras.plan_df[['plan_name', 'plan_num', 'plan_path']].to_string(index=False))

## 2. BASELINE: Extract Existing Results from Plan 02

First, extract and analyze the baseline breach behavior from the existing Plan 02 results.

### Important Note: HDF Results Files

**This example project may not include pre-computed HDF results** (.p02.hdf files). These files are generated when HEC-RAS runs a simulation.

**If you encounter "HDF file not found" errors:**
1. Option A: Run HEC-RAS simulation for Plan 02 first
2. Option B: Use the RasCmdr class to run simulation from Python (see cell below)
3. Option C: Use a different project with existing results (e.g., Scott County)

**The notebook will gracefully handle missing results and still demonstrate parameter modification.**

In [ ]:
# OPTIONAL: Run HEC-RAS Simulation for Plan 02
# Uncomment the lines below if you need to generate results

# from ras_commander import RasCmdr
# print("Running HEC-RAS simulation for Plan 02...")
# RasCmdr.compute_plan("02")
# print("Simulation complete! HDF results file created.")

# Initialize variables to None (prevents NameError in later cells)
target_structure = None
baseline_ts = pd.DataFrame()
baseline_summary = pd.DataFrame()
baseline_params = None
baseline_geom = []
scenarios = {}
summaries = {}

### 2.1 Identify Breach Structures

In [ ]:
# Try to list SA/2D connection structures in HDF results
try:
    hdf_structures = RasBreach.list_breach_structures_hdf("02")
    print("SA/2D Connection Structures in HDF:")
    for struct in hdf_structures:
        print(f"  - {struct}")

    # Get breach capability information
    breach_info = RasBreach.get_breach_info("02")
    print("\nBreach Capability Information:")
    print(breach_info.to_string(index=False))

    # Get list of structures with breach capability
    breach_structures = breach_info[breach_info['has_breach']]['structure'].tolist()
    print(f"\nStructures with breach capability: {breach_structures}")

    # Select first breach structure for analysis
    if breach_structures:
        target_structure = breach_structures[0]
        print(f"\nTarget structure for analysis: {target_structure}")
    else:
        print("\nWARNING: No breach structures found! Cannot proceed with analysis.")
        target_structure = None
        
except FileNotFoundError as e:
    print(f"\n⚠️ HDF RESULTS FILE NOT FOUND")
    print(f"Error: {e}")
    print(f"\nThis means HEC-RAS has not been run for Plan 02 yet.")
    print(f"To generate results, either:")
    print(f"  1. Uncomment and run the simulation cell above, OR")
    print(f"  2. Open HEC-RAS and manually run Plan 02")
    print(f"\nThe notebook will continue to demonstrate parameter modification.")
    target_structure = None
    breach_structures = []
except Exception as e:
    print(f"\n⚠️ UNEXPECTED ERROR: {e}")
    import traceback
    traceback.print_exc()
    target_structure = None
    breach_structures = []

### 2.2 Extract Baseline Time Series

In [ ]:
if target_structure:
    try:
        # Extract complete breach time series
        baseline_ts = RasBreach.get_breach_timeseries("02", target_structure)
        
        print(f"Baseline Time Series Extracted: {baseline_ts.shape}")
        print(f"\nColumns: {list(baseline_ts.columns)}")
        print(f"\nFirst few timesteps:")
        print(baseline_ts.head())
        
        # Get summary statistics
        baseline_summary = RasBreach.get_breach_summary("02", target_structure)
        print(f"\nBaseline Summary Statistics:")
        print(baseline_summary.to_string(index=False))
        
        # Store baseline for later comparison
        scenarios = {
            'Baseline (Plan 02)': baseline_ts.copy()
        }
        summaries = {
            'Baseline (Plan 02)': baseline_summary.copy()
        }
    except Exception as e:
        print(f"Could not extract baseline time series: {e}")
        print("Continuing with parameter analysis only...")
        baseline_ts = pd.DataFrame()
        baseline_summary = pd.DataFrame()
        scenarios = {}
        summaries = {}
else:
    print("Skipping baseline extraction - no breach structure available")
    scenarios = {}
    summaries = {}

### 2.3 Read Baseline Parameters

In [ ]:
if target_structure:
    try:
        # Read breach parameters from plan file
        baseline_params = RasBreach.read_breach_block("02", target_structure)
        
        print(f"Baseline Parameters for {target_structure}:")
        print("=" * 80)
        print(f"\nActivation: {baseline_params['is_active']}")
        print(f"\nKey Parameter Values:")
        for key in ['Breach Method', 'Breach Geom', 'Breach Start', 'Breach Progression']:
            if key in baseline_params['values']:
                print(f"  {key}: {baseline_params['values'][key]}")
        
        # Parse geometry values for modification
        geom_str = baseline_params['values'].get('Breach Geom', '')
        baseline_geom = [x.strip() for x in geom_str.split(',') if x.strip()]
        print(f"\nBaseline Geometry (parsed): {baseline_geom}")
    except Exception as e:
        print(f"Could not read baseline parameters: {e}")
        baseline_params = None
        baseline_geom = []
else:
    print("Skipping parameter reading - no breach structure available")
    baseline_params = None
    baseline_geom = []

### 2.4 Visualize Baseline Results

In [ ]:
if target_structure and not baseline_ts.empty:
    fig, axes = plt.subplots(3, 1, figsize=(12, 10))
    
    # Flow hydrograph
    axes[0].plot(baseline_ts['datetime'], baseline_ts['total_flow'], 
                'b-', linewidth=2, label='Total Flow')
    if baseline_ts['breach_flow'].notna().any():
        axes[0].plot(baseline_ts['datetime'], baseline_ts['breach_flow'], 
                    'r-', linewidth=2, label='Breach Flow')
    axes[0].set_ylabel('Flow (cfs)', fontsize=11)
    axes[0].set_title(f'BASELINE - {target_structure} Flow Hydrograph', 
                     fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Water surface elevations
    axes[1].plot(baseline_ts['datetime'], baseline_ts['hw'], 
                'b-', linewidth=2, label='Headwater')
    axes[1].plot(baseline_ts['datetime'], baseline_ts['tw'], 
                'r-', linewidth=2, label='Tailwater')
    axes[1].set_ylabel('Elevation (ft)', fontsize=11)
    axes[1].set_title('Water Surface Elevations', fontsize=12, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # Breach width (if available)
    if baseline_ts['bottom_width'].notna().any():
        axes[2].plot(baseline_ts['datetime'], baseline_ts['bottom_width'], 
                    'g-', linewidth=2, marker='o', markersize=4)
        axes[2].set_ylabel('Width (ft)', fontsize=11)
    else:
        axes[2].text(0.5, 0.5, 'Breach Not Formed', 
                    ha='center', va='center', fontsize=14,
                    transform=axes[2].transAxes)
    axes[2].set_title('Breach Width Evolution', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('Time', fontsize=11)
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("Skipping baseline visualization - no data available")

## 3. SCENARIO ANALYSIS: Modify Parameters and Compare Results

Now we'll create multiple scenarios by modifying breach parameters one at a time.

**Workflow for each scenario:**
1. Clone Plan 02 to a new plan number
2. Modify ONE parameter in the cloned plan
3. **[User must run HEC-RAS simulation]**
4. Extract results from the new plan
5. Compare with baseline

**Note:** This notebook demonstrates steps 1-2 and 4-5. You must run HEC-RAS (step 3) separately.

### Helper Function: Clone Plan File

In [ ]:
if target_structure and baseline_geom and len(baseline_geom) >= 2:
    # Clone plan
    plan_num_1 = "03"
    clone_plan("02", plan_num_1, "Scenario 1: +50% Width")
    
    # Modify breach width (assuming index 1 is width)
    try:
        new_geom = baseline_geom.copy()
        original_width = float(baseline_geom[1])
        new_width = original_width * 1.5
        new_geom[1] = new_width
        
        print(f"\nModifying breach width:")
        print(f"  Original: {original_width} ft")
        print(f"  New: {new_width} ft (+50%)")
        
        # Update the plan
        RasBreach.update_breach_block(
            plan_num_1,
            target_structure,
            geom_values=new_geom
        )
        
        print(f"\n✓ Scenario 1 plan created: {plan_num_1}")
        print(f"  Next step: Run HEC-RAS simulation for plan {plan_num_1}")
    except (ValueError, IndexError) as e:
        print(f"Could not parse geometry values: {e}")
else:
    print("Skipping Scenario 1 - insufficient baseline data")

### Scenario 1: Increase Breach Width by 50%

In [ ]:
if target_structure and baseline_geom and len(baseline_geom) >= 7:
    # Clone plan
    plan_num_2 = "04"
    clone_plan("02", plan_num_2, "Scenario 2: -50% Formation Time")
    
    # Modify formation time (assuming index 6 is formation time)
    try:
        new_geom = baseline_geom.copy()
        original_time = float(baseline_geom[6])
        new_time = original_time * 0.5
        new_geom[6] = new_time
        
        print(f"\nModifying breach formation time:")
        print(f"  Original: {original_time} hrs")
        print(f"  New: {new_time} hrs (-50%)")
        
        # Update the plan
        RasBreach.update_breach_block(
            plan_num_2,
            target_structure,
            geom_values=new_geom
        )
        
        print(f"\n✓ Scenario 2 plan created: {plan_num_2}")
        print(f"  Next step: Run HEC-RAS simulation for plan {plan_num_2}")
    except (ValueError, IndexError) as e:
        print(f"Could not parse geometry values: {e}")
else:
    print("Skipping Scenario 2 - insufficient baseline data")

### Scenario 2: Decrease Breach Formation Time by 50%

In [ ]:
if target_structure and baseline_params:
    # Clone plan
    plan_num_3 = "05"
    clone_plan("02", plan_num_3, "Scenario 3: Different Method")
    
    # Get current method
    current_method = int(baseline_params['values'].get('Breach Method', '0').strip())
    new_method = 1 if current_method == 0 else 0  # Toggle method
    
    print(f"\nModifying breach method:")
    print(f"  Original: {current_method}")
    print(f"  New: {new_method}")
    
    # Update the plan
    RasBreach.update_breach_block(
        plan_num_3,
        target_structure,
        method=new_method
    )
    
    print(f"\n✓ Scenario 3 plan created: {plan_num_3}")
    print(f"  Next step: Run HEC-RAS simulation for plan {plan_num_3}")
else:
    print("Skipping Scenario 3 - no baseline parameters available")

### Scenario 3: Change Breach Method

In [ ]:
if target_structure:
    # Clone plan
    plan_num_3 = "05"
    clone_plan("02", plan_num_3, "Scenario 3: Different Method")
    
    # Get current method
    current_method = int(baseline_params['values'].get('Breach Method', '0').strip())
    new_method = 1 if current_method == 0 else 0  # Toggle method
    
    print(f"\nModifying breach method:")
    print(f"  Original: {current_method}")
    print(f"  New: {new_method}")
    
    # Update the plan
    RasBreach.update_breach_block(
        plan_num_3,
        target_structure,
        method=new_method
    )
    
    print(f"\n✓ Scenario 3 plan created: {plan_num_3}")
    print(f"  Next step: Run HEC-RAS simulation for plan {plan_num_3}")
else:
    print("Skipping Scenario 3 - no breach structure available")

## 4. Extract Results from All Scenarios

**⚠️ IMPORTANT:** You must run HEC-RAS simulations for plans 03, 04, and 05 before this section will work.

This section attempts to extract results from all scenarios. If HDF files don't exist, it will skip gracefully.

In [ ]:
if target_structure:
    # Scenario definitions
    scenario_plans = {
        'Scenario 1: +50% Width': '03',
        'Scenario 2: -50% Formation Time': '04',
        'Scenario 3: Different Method': '05'
    }
    
    # Try to extract results for each scenario
    for scenario_name, plan_num in scenario_plans.items():
        try:
            # Check if HDF exists
            plan_row = ras.plan_df[ras.plan_df['plan_num'] == plan_num]
            if not plan_row.empty:
                ts = RasBreach.get_breach_timeseries(plan_num, target_structure)
                summary = RasBreach.get_breach_summary(plan_num, target_structure)
                
                if not ts.empty:
                    scenarios[scenario_name] = ts
                    summaries[scenario_name] = summary
                    print(f"✓ Extracted: {scenario_name}")
                else:
                    print(f"⚠ No results for: {scenario_name} (run HEC-RAS first)")
            else:
                print(f"⚠ Plan {plan_num} not found for: {scenario_name}")
        except Exception as e:
            print(f"⚠ Could not extract {scenario_name}: {e}")
    
    print(f"\nTotal scenarios with results: {len(scenarios)}")
else:
    print("Skipping scenario extraction - no breach structure available")

## 5. Compare Results Across All Scenarios

Visualize all scenarios together to understand parameter sensitivity.

### 5.1 Flow Hydrograph Comparison

In [ ]:
if scenarios:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    linestyles = ['-', '--', '-.', ':', '-']
    
    for idx, (scenario_name, ts_data) in enumerate(scenarios.items()):
        color = colors[idx % len(colors)]
        linestyle = linestyles[idx % len(linestyles)]
        
        ax.plot(ts_data['datetime'], ts_data['total_flow'],
               label=scenario_name, color=color, linestyle=linestyle, linewidth=2)
    
    ax.set_xlabel('Time', fontsize=12)
    ax.set_ylabel('Total Flow (cfs)', fontsize=12)
    ax.set_title(f'{target_structure} - Flow Hydrograph Comparison', 
                fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No scenario data available for comparison")

### 5.2 Peak Flow Comparison (Bar Chart)

In [ ]:
if summaries:
    # Extract peak flows
    scenario_names = list(summaries.keys())
    peak_flows = [summaries[name].iloc[0]['max_total_flow'] 
                 for name in scenario_names]
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    bars = ax.bar(range(len(scenario_names)), peak_flows, 
                  color=['blue', 'red', 'green', 'orange', 'purple'][:len(scenario_names)])
    
    ax.set_xticks(range(len(scenario_names)))
    ax.set_xticklabels(scenario_names, rotation=45, ha='right')
    ax.set_ylabel('Peak Flow (cfs)', fontsize=12)
    ax.set_title(f'{target_structure} - Peak Flow Comparison', 
                fontsize=14, fontweight='bold')
    ax.grid(True, axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars, peak_flows):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
               f'{value:.0f}',
               ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Print percent differences from baseline
    if len(peak_flows) > 1:
        baseline_flow = peak_flows[0]
        print("\nPeak Flow Differences from Baseline:")
        print("=" * 60)
        for i, (name, flow) in enumerate(zip(scenario_names, peak_flows)):
            if i == 0:
                print(f"{name}: {flow:.0f} cfs (baseline)")
            else:
                diff_pct = ((flow - baseline_flow) / baseline_flow) * 100
                print(f"{name}: {flow:.0f} cfs ({diff_pct:+.1f}%)")
else:
    print("No summary data available for comparison")

### 5.3 Breach Width Evolution Comparison

In [ ]:
if scenarios:
    # Check if any scenario has breach width data
    has_width_data = any(ts['bottom_width'].notna().any() for ts in scenarios.values())
    
    if has_width_data:
        fig, ax = plt.subplots(figsize=(14, 6))
        
        for idx, (scenario_name, ts_data) in enumerate(scenarios.items()):
            if ts_data['bottom_width'].notna().any():
                color = colors[idx % len(colors)]
                linestyle = linestyles[idx % len(linestyles)]
                
                ax.plot(ts_data['datetime'], ts_data['bottom_width'],
                       label=scenario_name, color=color, linestyle=linestyle, 
                       linewidth=2, marker='o', markersize=4)
        
        ax.set_xlabel('Time', fontsize=12)
        ax.set_ylabel('Breach Width (ft)', fontsize=12)
        ax.set_title(f'{target_structure} - Breach Width Evolution Comparison', 
                    fontsize=14, fontweight='bold')
        ax.legend(loc='best', fontsize=10)
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    else:
        print("No breach width data available (breach may not have formed)")
else:
    print("No scenario data available for comparison")

### 5.4 Summary Table Comparison

In [ ]:
if summaries:
    # Combine all summaries into a comparison table
    comparison_data = []
    for scenario_name, summary_df in summaries.items():
        row = summary_df.iloc[0].to_dict()
        row['Scenario'] = scenario_name
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Select key columns for display
    display_cols = ['Scenario', 'max_total_flow', 'max_breach_flow', 
                   'final_breach_width', 'final_breach_depth', 
                   'max_hw', 'max_tw']
    
    # Filter to available columns
    display_cols = [col for col in display_cols if col in comparison_df.columns]
    
    print("\nScenario Comparison Summary:")
    print("=" * 100)
    print(comparison_df[display_cols].to_string(index=False))
    
    # Export to CSV
    output_file = project_path / "breach_scenario_comparison.csv"
    comparison_df.to_csv(output_file, index=False)
    print(f"\nComparison table exported to: {output_file}")
else:
    print("No summary data available for comparison table")

### 5.5 Comprehensive Multi-Panel Comparison

In [ ]:
if scenarios and len(scenarios) > 1:
    # Create 2x2 subplot grid
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: Flow comparison
    for idx, (scenario_name, ts_data) in enumerate(scenarios.items()):
        color = colors[idx % len(colors)]
        axes[0, 0].plot(ts_data['datetime'], ts_data['total_flow'],
                       label=scenario_name, color=color, linewidth=2)
    axes[0, 0].set_ylabel('Total Flow (cfs)', fontsize=11)
    axes[0, 0].set_title('Flow Hydrographs', fontsize=12, fontweight='bold')
    axes[0, 0].legend(fontsize=8)
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Peak flows bar chart
    scenario_names = list(summaries.keys())
    peak_flows = [summaries[name].iloc[0]['max_total_flow'] for name in scenario_names]
    bars = axes[0, 1].bar(range(len(scenario_names)), peak_flows,
                          color=colors[:len(scenario_names)])
    axes[0, 1].set_xticks(range(len(scenario_names)))
    axes[0, 1].set_xticklabels(range(len(scenario_names)))
    axes[0, 1].set_ylabel('Peak Flow (cfs)', fontsize=11)
    axes[0, 1].set_title('Peak Flow Comparison', fontsize=12, fontweight='bold')
    axes[0, 1].grid(True, axis='y', alpha=0.3)
    
    # Plot 3: HW/TW for baseline
    baseline_ts = scenarios[list(scenarios.keys())[0]]
    axes[1, 0].plot(baseline_ts['datetime'], baseline_ts['hw'], 
                   label='HW', color='blue', linewidth=2)
    axes[1, 0].plot(baseline_ts['datetime'], baseline_ts['tw'], 
                   label='TW', color='red', linewidth=2)
    axes[1, 0].set_xlabel('Time', fontsize=11)
    axes[1, 0].set_ylabel('Elevation (ft)', fontsize=11)
    axes[1, 0].set_title('Baseline Water Levels', fontsize=12, fontweight='bold')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Breach width comparison (if available)
    has_width = False
    for idx, (scenario_name, ts_data) in enumerate(scenarios.items()):
        if ts_data['bottom_width'].notna().any():
            color = colors[idx % len(colors)]
            axes[1, 1].plot(ts_data['datetime'], ts_data['bottom_width'],
                           label=scenario_name, color=color, linewidth=2, marker='o')
            has_width = True
    
    if has_width:
        axes[1, 1].set_xlabel('Time', fontsize=11)
        axes[1, 1].set_ylabel('Breach Width (ft)', fontsize=11)
        axes[1, 1].set_title('Breach Width Evolution', fontsize=12, fontweight='bold')
        axes[1, 1].legend(fontsize=8)
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'No Breach Width Data',
                       ha='center', va='center', fontsize=14,
                       transform=axes[1, 1].transAxes)
    
    fig.suptitle(f'{target_structure} - Breach Scenario Analysis Dashboard',
                fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()
else:
    print("Need at least 2 scenarios for comprehensive comparison")

## 6. Export All Results

In [ ]:
if scenarios:
    # Export each scenario's time series
    for scenario_name, ts_data in scenarios.items():
        # Create safe filename
        safe_name = scenario_name.replace(' ', '_').replace(':', '').replace('+', 'plus')
        filename = project_path / f"breach_{safe_name}.csv"
        ts_data.to_csv(filename, index=False)
        print(f"Exported: {filename.name}")
    
    print(f"\nAll scenario data exported to: {project_path}")
else:
    print("No scenario data to export")

## Summary

This notebook demonstrated:

**✅ Baseline Analysis:**
- Extracted existing breach results from Plan 02
- Read baseline breach parameters
- Visualized baseline behavior

**✅ Scenario Creation:**
- Cloned Plan 02 to create new scenarios
- Modified breach parameters one at a time:
  - Scenario 1: Increased breach width by 50%
  - Scenario 2: Decreased formation time by 50%
  - Scenario 3: Changed breach method

**✅ Results Comparison:**
- Extracted results from all scenarios (if HEC-RAS was run)
- Compared flow hydrographs
- Compared peak flows
- Compared breach geometry evolution
- Created comprehensive comparison dashboard

**✅ Data Export:**
- Exported time series for all scenarios
- Exported comparison summary table

### Next Steps:

1. **Run HEC-RAS Simulations:**
   - Open HEC-RAS
   - Run plans 03, 04, and 05
   - Re-run this notebook to extract and compare results

2. **Additional Scenarios:**
   - Create more scenarios by modifying other parameters
   - Test combinations of parameters
   - Explore full sensitivity range

3. **Advanced Analysis:**
   - Statistical analysis of parameter sensitivity
   - Uncertainty quantification
   - Flood impact assessment downstream

### Key RasBreach Functions Used:

```python
# HDF Extraction
RasBreach.list_breach_structures_hdf(plan)
RasBreach.get_breach_info(plan)
RasBreach.get_breach_timeseries(plan, structure)
RasBreach.get_breach_summary(plan, structure)

# Plan File Management
RasBreach.list_breach_structures_plan(plan)
RasBreach.read_breach_block(plan, structure)
RasBreach.update_breach_block(plan, structure, **params)
```